# Genetic Algorithm Sudoku Solver
⭐ Idea

Instead of using backtracking or constraint logic, we treat each Sudoku grid as an organism, and evolve a population until one becomes a valid solution.

Main components:

Representation: A 9×9 grid where each 3×3 block contains digits 1–9 (keeps structure valid).

Fitness function: Counts duplicates in rows/columns (lower is better).

Mutation: Swap two numbers inside a 3×3 block.

Crossover: Choose blocks from each parent.

Selection: Take fittest individuals for breeding.


## Full Python Code (Compact & Understandable)

In [2]:
import random
import copy

# ---- CONFIG ----
POP_SIZE = 200
GENERATIONS = 2000
MUTATION_RATE = 0.1

# Example Sudoku puzzle (0 = empty)
PUZZLE = [
    [5,3,0, 0,7,0, 0,0,0],
    [6,0,0, 1,9,5, 0,0,0],
    [0,9,8, 0,0,0, 0,6,0],

    [8,0,0, 0,6,0, 0,0,3],
    [4,0,0, 8,0,3, 0,0,1],
    [7,0,0, 0,2,0, 0,0,6],

    [0,6,0, 0,0,0, 2,8,0],
    [0,0,0, 4,1,9, 0,0,5],
    [0,0,0, 0,8,0, 0,7,9],
]


# ---- GENETIC ALGORITHM IMPLEMENTATION ----

def create_candidate(puzzle):
    """Fill each 3x3 block randomly respecting fixed clues."""
    candidate = copy.deepcopy(puzzle)
    for br in range(3):
        for bc in range(3):
            existing = {candidate[r][c]
                        for r in range(br*3, br*3+3)
                        for c in range(bc*3, bc*3+3)
                        if candidate[r][c] != 0}

            missing = list(set(range(1,10)) - existing)
            random.shuffle(missing)

            for r in range(br*3, br*3+3):
                for c in range(bc*3, bc*3+3):
                    if candidate[r][c] == 0:
                        candidate[r][c] = missing.pop()
    return candidate


def fitness(grid):
    """Count duplicates in rows & columns."""
    score = 0
    for i in range(9):
        score += (9 - len(set(grid[i])))                  # row duplicates
        score += (9 - len(set(row[i] for row in grid)))   # column duplicates
    return score


def mutate(grid, puzzle):
    """Swap two random cells inside a block, but only if not fixed clues."""
    if random.random() > MUTATION_RATE:
        return grid

    r = random.randint(0,8)
    c = random.randint(0,8)
    block_r = (r // 3) * 3
    block_c = (c // 3) * 3

    # Find mutable cells in the same block
    mutable = [(i,j) for i in range(block_r, block_r+3)
                      for j in range(block_c, block_c+3)
                      if puzzle[i][j] == 0]

    if len(mutable) >= 2:
        a, b = random.sample(mutable, 2)
        grid[a[0]][a[1]], grid[b[0]][b[1]] = grid[b[0]][b[1]], grid[a[0]][a[1]]

    return grid


def crossover(parent1, parent2):
    """Combine parents block-by-block into a correct 9×9 child."""
    child = [[0]*9 for _ in range(9)]
    
    for br in range(3):
        for bc in range(3):
            # choose block
            src = parent1 if random.random() < 0.5 else parent2
            
            for r in range(3):
                for c in range(3):
                    child[br*3 + r][bc*3 + c] = src[br*3 + r][bc*3 + c]
    return child


# ---- MAIN EVOLUTION LOOP ----

population = [create_candidate(PUZZLE) for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    population.sort(key=fitness)
    best = population[0]

    if fitness(best) == 0:
        print(f"Solved in generation {gen}!")
        break

    next_pop = population[:20]  # keep best 20

    # Breed new candidates
    while len(next_pop) < POP_SIZE:
        p1, p2 = random.sample(population[:50], 2)
        child = crossover(p1, p2)
        child = mutate(child, PUZZLE)
        next_pop.append(child)

    population = next_pop

# ---- OUTPUT ----
for row in best:
    print(row)


[5, 3, 4, 6, 7, 8, 1, 9, 2]
[6, 2, 7, 1, 9, 5, 4, 3, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 2, 9, 6, 1, 7, 4, 3]
[4, 1, 6, 8, 5, 3, 9, 2, 1]
[7, 3, 9, 7, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[3, 8, 7, 4, 1, 9, 6, 1, 5]
[2, 4, 5, 2, 8, 6, 3, 7, 9]
